# Question 4: Implement ReAct Agent with Multiple Tools (25 points)

Implement a ReAct (Reasoning and Acting) agent as described by Yao et al. [1], incorporating three main tools: search, compare, and analyze. This agent should be able to handle complex queries by reasoning about which tool to use and when.

a) (4 points) Implement the search tool using the SerpAPI integration from previous questions. Ensure it can be easily used by the ReAct agent.
   - Proper integration with SerpAPI
   - Formatting the search results for use by the ReAct agent

b) (5 points) Create a custom comparison tool using LangChain's `Tool` class. The tool should accept multiple items and a category as input and return a comparison result.
   - Implementing the comparison logic
   - Creating an appropriate prompt template for the comparison
   - Proper error handling for invalid inputs

c) (5 points) Implement an analysis tool that can summarize and extract key information from search results or comparisons. This tool should use the OpenAI model to generate insightful analyses.
   - Implementing the analysis logic
   - Creating an appropriate prompt template for the analysis
   - Ensuring the analysis output is concise and relevant

d) (6 points) Integrate these tools with a ReAct agent using LangChain. Your implementation should:
   - Use LangChain's `initialize_agent` function with the `AgentType.ZERO_SHOT_REACT_DESCRIPTION` agent type
   - Include all three tools (search, compare, analyze) as available actions for the agent
   - Implement proper error handling and fallback strategies
   - Ensure smooth transitions between tools in the agent's reasoning process

e) (5 points) Implement a simple Streamlit user interface for your ReAct agent. Your implementation should include:
   - A text input field for users to enter their queries
   - A button to submit the query and trigger the ReAct agent
   - A display area for showing the final results
   - A section to display the step-by-step reasoning process of the ReAct agent

In [ ]:
# Install required packages
# Pinned versions verified to work with this notebook
!pip install \
    "langchain==0.3.25" \
    "langchain-community==0.3.24" \
    "langchain-core==0.3.62" \
    "langchain-openai==0.2.14" \
    "google-search-results==2.4.2" \
    "streamlit>=1.32.0"

In [ ]:
# Import necessary libraries
import os
from google.colab import userdata
from langchain_openai import OpenAI
from langchain_core.tools import Tool                                          # langchain_core >= 0.1
from langchain_community.agent_toolkits.load_tools import load_tools          # langchain_community >= 0.0.38
from langchain.agents import initialize_agent, AgentType
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Set API keys
# TODO: Retrieve your API keys from Colab secrets and set them as environment variables
os.environ["OPENAI_API_KEY"]  = userdata.get("OPENAI_API_KEY")
os.environ["SERPAPI_API_KEY"] = userdata.get("SERPAPI_API_KEY")

# Initialize the OpenAI language model (Use temp=0)
# TODO: Create an OpenAI instance with temperature=0
llm = ...

## a) Implement the search tool

In [ ]:
# Load the search tool using SerpAPI
# Hint: Use load_tools(["serpapi"], llm=llm) to get a LangChain-wrapped SerpAPI tool.
# The returned list contains a ready-to-use Tool object — store it as `search_tool`.

# TODO: Load the SerpAPI search tool
search_tool = ...

## b) Create a custom comparison tool

In [ ]:
def compare_items(query: str) -> str:
    """Compare multiple items in a given category.

    The tool expects `query` to follow the format:
        "item1, item2, ..., category"
    For example: "iPhone 15 Pro, Samsung Galaxy S24 Ultra, smartphones"

    Returns a string summarising the comparison, or an error message
    if the input cannot be parsed.
    """
    try:
        # TODO: Split `query` on commas to extract individual tokens.
        # The last token is the category; all preceding tokens are the items.
        parts = ...
        if len(parts) < 3:
            return "Error: please provide at least two items and a category."
        items = ...       # all tokens except the last
        category = ...    # the last token

        # TODO: Create a PromptTemplate that asks the LLM to compare the items
        # within the given category. Include {items} and {category} as variables.
        comparison_template = """..."""
        comparison_prompt = PromptTemplate(
            input_variables=["items", "category"],
            template=comparison_template
        )

        # TODO: Build an LLMChain and call .invoke() to run it.
        # .invoke() takes a dict of template variables and returns a dict;
        # access the generated text with ["text"].
        comparison_chain = LLMChain(llm=llm, prompt=comparison_prompt)
        result = comparison_chain.invoke({"items": ..., "category": ...})["text"]
        return result.strip()

    except Exception as e:
        return f"Error in compare_items: {str(e)}"


# TODO: Wrap compare_items in a LangChain Tool object.
# Give it a clear name and description so the ReAct agent knows when to use it.
compare_tool = Tool(
    name=...,
    func=compare_items,
    description=...
)

## c) Implement an analysis tool

In [ ]:
def analyze_results(query: str) -> str:
    """Summarize and extract key information from text passed by the agent.

    Note: LangChain Tool functions must accept a *single* string argument.
    The agent will pass whatever text it wants analyzed directly as `query`.

    Returns a concise, insightful analysis string.
    """
    # TODO: Create a PromptTemplate that instructs the LLM to produce a concise
    # summary and highlight the key takeaways from the provided text.
    # Use {text} as the only template variable.
    analysis_template = """..."""
    analysis_prompt = PromptTemplate(
        input_variables=["text"],
        template=analysis_template
    )

    # TODO: Build an LLMChain and call .invoke() to run it.
    # .invoke() takes a dict of template variables and returns a dict;
    # access the generated text with ["text"].
    analysis_chain = LLMChain(llm=llm, prompt=analysis_prompt)
    result = analysis_chain.invoke({"text": query})["text"]
    return result.strip()


# TODO: Wrap analyze_results in a LangChain Tool object.
analyze_tool = Tool(
    name=...,
    func=analyze_results,
    description=...
)

## d) Integrate tools with a ReAct agent

In [ ]:
# TODO: Build the list of tools for the ReAct agent.
# Include search_tool, compare_tool, and analyze_tool.
tools = [...]

# TODO: Initialize the ReAct agent using initialize_agent().
# Use AgentType.ZERO_SHOT_REACT_DESCRIPTION and set verbose=True so you can
# observe the Thought / Action / Observation trace in the output.
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=...,
    verbose=...,
    max_iterations=...,   # guard against infinite loops
    handle_parsing_errors=True
)

In [ ]:
def process_query(query: str, max_steps: int = 100) -> str:
    """Run the ReAct agent on `query` and return its final answer.

    Args:
        query: The natural-language question to answer.
        max_steps: Maximum number of reasoning steps before stopping.

    Returns:
        The agent's final answer as a string, or an error message.
    """
    try:
        # TODO: Call agent.invoke({"input": query}) and return the final answer.
        # .invoke() returns a dict; access the answer with ["output"].
        # The agent object was created in the cell above.
        output = agent.invoke({"input": query})
        return output["output"]
    except Exception as e:
        return f"Error processing query: {str(e)}"

## e) Streamlit User Interface

Create a separate Python file (e.g. `app.py`) containing a Streamlit app for your ReAct agent. Your app must include:

- A **text input** field for the user's query
- A **submit button** to trigger the agent
- A **results area** that displays the final answer
- *(Optional)* A collapsible section showing the step-by-step Thought / Action / Observation trace

> **Note:** Streamlit apps must be run from the terminal with `streamlit run app.py`. You can use `%%writefile app.py` in a code cell to write the file directly from this notebook.

In [ ]:
%%writefile app.py
import os
import streamlit as st
from langchain_openai import OpenAI
from langchain_core.tools import Tool                                          # langchain_core >= 0.1
from langchain_community.agent_toolkits.load_tools import load_tools          # langchain_community >= 0.0.38
from langchain.agents import initialize_agent, AgentType
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# TODO: Import and configure your LLM, tools, and agent here.
# Use st.secrets["OPENAI_API_KEY"] and st.secrets["SERPAPI_API_KEY"]
# to read keys when running outside Colab.

st.title("ReAct Agent")
st.write("Ask a complex question and let the agent reason through it step by step.")

# TODO: Add a text input widget for the user's query.
query = st.text_input(...)

# TODO: Add a button to submit the query.
if st.button(...):
    if query:
        with st.spinner("Thinking..."):
            # TODO: Call process_query() (or agent.invoke()) and store the result.
            result = ...

        # TODO: Display the final answer.
        st.subheader("Answer")
        st.write(result)

        # (Optional) Display the step-by-step reasoning trace.
        # Hint: use return_intermediate_steps=True on initialize_agent, then
        # iterate over output["intermediate_steps"] to show each action + observation.
    else:
        st.warning("Please enter a query before submitting.")

## Test Your Implementation

Use the cell below to test your implementation with a sample query.

In [ ]:
# Test your implementation
sample_query = "What are the top 3 smartphones in 2023, and how do they compare in terms of camera quality and battery life?"

result = process_query(sample_query)
print(result)

## Submission Requirements

Please submit the following items as part of your solution:

1. Your complete code implementation for the ReAct agent and its tools (this notebook).
2. Your `app.py` Streamlit file (Part e).
3. A sample question that you used to test your tool (make it complex enough to demonstrate the use of multiple tools).
4. The final answer provided by your ReAct agent for the sample question.
5. The complete history traces of the ReAct agent for your sample question, showing its thought process, actions, and observations. Your traces should follow a format similar to this example:

```
Thought: I need to find information about top smartphones first
Action: Search[top smartphones 2023]
Observation: [Search results about top smartphones]
Thought: Now I should compare the top two options
Action: Compare[iPhone 14 Pro, Samsung Galaxy S23 Ultra, smartphones]
Observation: [Comparison result]
Thought: I should analyze this comparison for the user
Action: Analyze[comparison result]
Observation: [Analysis of the comparison]
Final Answer: [Your agent's final response to the user's query]
```

Ensure that your submission clearly demonstrates the agent's ability to reason about which tool to use and how to interpret the results from each tool. Your history traces should show a logical flow of thoughts, actions, and observations, culminating in a final answer that addresses the initial query.

**Note:** Ensure that your ReAct agent can seamlessly switch between these tools based on the task at hand. The agent should be able to reason about which tool to use next and how to interpret the results from each tool.

## References

[1] Yao, S., Zhao, J., Yu, D., Du, N., Shafran, I., Narasimhan, K., & Cao, Y. (2022). ReAct: Synergizing reasoning and acting in language models. arXiv preprint arXiv:2210.03629. https://arxiv.org/pdf/2210.03629